In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import numpy as np
from pandas.tseries.offsets import BusinessDay, MonthEnd, MonthBegin

In [2]:
#=========
# Setup
#=========
ts = pd.DataFrame({
    "date": pd.to_datetime(["2025-12-01" , "2025-12-02" , "2025-12-04" , "2025-12-07"]) ,
    "sales": [120 , 125 , 110 , 140] ,
}).set_index("date").sort_index()
ts

,sales
date,
2025-12-01,120
2025-12-02,125
2025-12-04,110
2025-12-07,140


In [5]:
#==================================
# Case 1) Shift by calendar time
#==================================
ts["sales_prev_7D"] = ts["sales"].shift(freq = "7D")
ts

,sales,sales_prev_7D
date,,
2025-12-01,120,NaN
2025-12-02,125,NaN
2025-12-04,110,NaN
2025-12-07,140,NaN


In [8]:
#========================================================
# Case 2) Business day ranges + business-day shifting
#========================================================
biz_days = pd.bdate_range("2025-12-01" , periods = 7)
d = pd.Timestamp("2025-12-05")
next_biz = d + BusinessDay(1)
print("\nBusiness days + BusinessDay offset:")
print("bdate_range:" , list(biz_days.strftime("%Y-%m-%d")))
print("Next business day after 2025-12-05:" , next_biz.strftime("%Y-%m-%d"))


Business days + BusinessDay offset:
bdate_range: ['2025-12-01', '2025-12-02', '2025-12-03', '2025-12-04', '2025-12-05', '2025-12-08', '2025-12-09']
Next business day after 2025-12-05: 2025-12-08


In [9]:
#=========================================
# Case 3) MonthEnd / MonthBegin offsets
#=========================================
x = pd.Timestamp("2025-12-17")
month_end = x + MonthEnd(0)
next_month_begin = x + MonthBegin(1)
print("Month end:" , month_end.strftime("%Y-%m-%d"))
print("Next month begin:" , next_month_begin.strftime("%Y-%m-%d"))

Month end: 2025-12-31
Next month begin: 2026-01-01


In [10]:
#================================================
# Setup log-like data for time-of-day filters
#================================================
log = pd.DataFrame({
    "ts": pd.to_datetime([
        "2025-12-01 08:30" , "2025-12-01 12:15" , "2025-12-01 18:05" ,
        "2025-12-02 09:05" , "2025-12-02 15:45" , "2025-12-02 20:10" ,
    ]) ,
    "events": [5 , 20 , 7 , 10 , 18 , 6] ,
}).set_index("ts").sort_index()
log

,events
ts,
2025-12-01 08:30:00,5
2025-12-01 12:15:00,20
2025-12-01 18:05:00,7
2025-12-02 09:05:00,10
2025-12-02 15:45:00,18
2025-12-02 20:10:00,6


In [11]:
#==========================================
# Case 4) between_time / at_time slicing
#==========================================
biz_hours = log.between_time("09:00" , "17:00")
biz_hours

daily_cutoff = log.at_time("12:15")
daily_cutoff

,events
ts,
2025-12-01 12:15:00,20
2025-12-02 09:05:00,10
2025-12-02 15:45:00,18


,events
ts,
2025-12-01 12:15:00,20


In [12]:
#=====================
# Case 5) truncate
#=====================
tr = log.truncate(before = "2025-12-01 12:00" , after = "2025-12-02 16:00")
tr

,events
ts,
2025-12-01 12:15:00,20
2025-12-01 18:05:00,7
2025-12-02 09:05:00,10
2025-12-02 15:45:00,18


In [13]:
#===================================================
# Case 6) merge_asof - nearest/previous time join
#===================================================
prices = pd.DataFrame({
    "ts": pd.to_datetime([
        "2025-12-01 09:00" , "2025-12-01 13:00" , "2025-12-02 10:00"
    ]) ,
    "price": [10.0 , 10.4 , 10.1] ,
}).sort_values("ts")
prices

trades = pd.DataFrame({
    "ts": pd.to_datetime([
        "2025-12-01 09:10" , "2025-12-01 12:50" , "2025-12-02 10:20"
    ]) ,
    "qty": [2 , 1 , 3] ,
}).sort_values("ts")
trades

joined = pd.merge_asof(trades , prices , on = "ts" , direction = "backward")
joined

,ts,price
0,2025-12-01 09:00:00,10.0
1,2025-12-01 13:00:00,10.4
2,2025-12-02 10:00:00,10.1


,ts,qty
0,2025-12-01 09:10:00,2
1,2025-12-01 12:50:00,1
2,2025-12-02 10:20:00,3


,ts,qty,price
0,2025-12-01 09:10:00,2,10.0
1,2025-12-01 12:50:00,1,10.0
2,2025-12-02 10:20:00,3,10.1
